# Tarea: Entendimiento de los datos

## Introducción
    Objetivo 

    ¿Para qué?


## Configuración e importe de paquetes

In [34]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType, StringType, IntegerType, DateType, TimestampType
from pyspark.sql.functions import udf, col, countDistinct, length, isnan, when, count, max as spark_max, to_date, year, to_timestamp, expr, substring, min, corr
import pyspark.sql.functions as f
import os 
from datetime import datetime
from pyspark.sql import types as t
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import numpy as np

In [2]:
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

In [3]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)

spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

C:\Users\estudiante\anaconda3\envs\Tutoriales\lib\site-packages\pyspark\sql\context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [4]:
db_connection_string = 'jdbc:mysql://157.253.236.116:8080/WWImportersTransactional'
db_user = 'xxx'
db_psswd = 'xxx'
PATH='./'

## Perfilamiento de los datos

### Información dada por la organización

- La cantidad máxima de productos movidos es 50 millones por transacción
- Existen 236.668 movimientos de producto realizados desde 2013
- Los movimientos están relacionados con 228.265 clientes
- El formato de fechas manejado es YYYY-MM-DD HH:MM:SS
- Actualmente tenemos 13 proveedores


### Conexión a fuente de datos y acceso a los datos

#### Conexión a fuente de datos

In [5]:
def obtener_dataframe_de_bd(db_connection_string, sql, db_user, db_psswd):
    df_bd = spark.read.format('jdbc')\
        .option('url', db_connection_string) \
        .option('dbtable', sql) \
        .option('user', db_user) \
        .option('password', db_psswd) \
        .option('driver', 'com.mysql.cj.jdbc.Driver') \
        .load()
    return df_bd

#### Cargue de datos
Acceso directo para el caso de movimientos y acceso con sentencia SQL para el caso de movimientosCopia.

In [6]:
sql_movimientos = 'WWImportersTransactional.movimientosCopia'

In [7]:
sql_detallesMovimientos = '''(SELECT 
        TransaccionProductoID,
        ProductoID,
        TipoTransaccionID,
        ClienteID,
        InvoiceID,
        ProveedorID,
        OrdenDeCompraID,
        FechaTransaccion,
        Cantidad 
        FROM WWImportersTransactional.movimientosCopia) AS Compatible'''

In [39]:
#Se cargan los dataframes desde la base de datos
movimientos = obtener_dataframe_de_bd(db_connection_string, sql_movimientos, db_user, db_psswd)

### Entendimiento general de datos

In [9]:
movimientos.show(7)

+---------------------+----------+-----------------+---------+---------+-----------+---------------+----------------+--------+
|TransaccionProductoID|ProductoID|TipoTransaccionID|ClienteID|InvoiceID|ProveedorID|OrdenDeCompraID|FechaTransaccion|Cantidad|
+---------------------+----------+-----------------+---------+---------+-----------+---------------+----------------+--------+
|               118903|       217|               10|    476.0|  24904.0|           |               |     Apr 25,2014|   -40.0|
|               286890|       135|               10|     33.0|  60117.0|           |               |     Dec 10,2015|    -7.0|
|               285233|       111|               10|    180.0|  59768.0|           |               |     Dec 04,2015|    -2.0|
|               290145|       213|               10|     33.0|  60795.0|           |               |     Dec 23,2015|    -3.0|
|               247492|        90|               10|     55.0|  51851.0|           |               |     Jul 27

In [11]:
movimientos.schema

StructType(List(StructField(TransaccionProductoID,IntegerType,true),StructField(ProductoID,IntegerType,true),StructField(TipoTransaccionID,IntegerType,true),StructField(ClienteID,DoubleType,true),StructField(InvoiceID,DoubleType,true),StructField(ProveedorID,StringType,true),StructField(OrdenDeCompraID,StringType,true),StructField(FechaTransaccion,StringType,true),StructField(Cantidad,DoubleType,true)))

#### Descripción de la Tabla **movimientosCopia**

La tabla registra los movimientos de productos en el inventario, se detalla la descripción de su estructura:

1. **TransaccionProductoID** (int): Identificador único de la transacción de producto.

2. **ProductoID** (int): Identificador del producto específico que se está moviendo en el inventario.

3. **TipoTransaccionID** (int): Identificador del tipo de transacción, indicando el tipo de movimiento de producto. Los tipos de transacciones son:
   - 2: Customer Credit Note
   - 3: Customer Payment Received
   - 4: Customer Refund
   - 5: Supplier Invoice
   - 6: Supplier Credit Note
   - 7: Supplier Payment Issued
   - 8: Supplier Refund
   - 9: Stock Transfer
   - 10: Stock Issue
   - 11: Stock Receipt
   - 12: Stock Adjustment at Stocktake
   - 13: Customer Contra

4. **ClienteID** (double): Identificador único del cliente asociado con la transacción. Esta columna puede tener valores nulos si el movimiento no está asociado a un cliente específico. Relacionada con la tabla **Clientes**.

5. **InvoiceID** (double): Identificador único de la factura asociada con la transacción, puede tener valores nulos.

6. **ProveedorID** (text): Identificador único del proveedor asociado con la transacción, puede tener valores nulos si el movimiento no está relacionado con un proveedor específico. Relacionada con la tabla **Proveedores**.

7. **OrdenDeCompraID** (text): Identificador de la orden de compra asociada con la transacción, puede tener valores nulos. Relacionada con la tabla **OrdenesCopia**.

8. **FechaTransaccion** (text): Fecha y hora en que se realizó la transacción. El formato de fecha manejado es YYYY-MM-DD HH:MM:SS.

9. **Cantidad** (double): Cantidad de productos movidos en la transacción, puede ser negativo, indicando una salida de inventario, o positivo, indicando una entrada.

#### Observaciones y consideraciones

1. **Valores nulos**: Varias columnas, como ClienteID, InvoiceID, ProveedorID y OrdenDeCompraID, pueden contener valores nulos. Es importante considerar cómo se deberían manejar estos valores en el análisis.

2. **Formato de fecha**: La columna FechaTransaccion almacena las fechas en formato de texto. Es importante convertir estas fechas al tipo de dato adecuado para realizar análisis temporales.

3. **Valores negativos en cantidad**: Los valores negativos en la columna Cantidad indican salidas de inventario. Información relevante para el análisis de inventario.

4. **Identificadores únicos**: TransaccionProductoID debe ser único para cada transacción, lo cual es crucial para identificar movimientos específicos sin ambigüedad.

5. **Relaciones entre columnas**: La tabla parece estar diseñada para poder relacionar transacciones con clientes, proveedores y órdenes de compra. Sin embargo, no todas las transacciones estarán necesariamente relacionadas con estos elementos.

In [12]:
print((movimientos.count(), len(movimientos.columns)))

(204292, 9)


**¿Cuantos datos nos compartieron para el proyecto?**
En total tenemos 204,292 movimientos y tenemos en total 9 variables a revisar de las cuales 6 son numéricas asociadas a las órdenes, una es de tipo fecha y las demás son cadenas de caracteres.

### Revisión de reglas de negocio
Primero se confirman las reglas proporcionadas por el negocio

#### Regla 1: La cantidad máxima de productos movidos es 50 millones por transacción

In [13]:
print(movimientos.agg(spark_max("Cantidad")).collect()[0][0])

67368.0


El análisis confirma que la cantidad máxima de productos movidos por transacción registrada en los datos es de 67,368, lo cual está dentro del límite aceptable de 50 millones. Por lo tanto, se puede concluir que los datos cumplen con esta regla de negocio específica. Este resultado contribuye a la confianza en la integridad y la calidad de los datos de movimientos de inventario de WWImporters.

#### Regla 2: Existen 236.668 movimientos de producto realizados desde eñ año 2013

In [14]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
# Extraer el año considerando teniendo en cuenta que se tien la fecha en multiples formatos
movimientosAnio = movimientos.withColumn(
    "AnioTransaccion",
    when(col("FechaTransaccion").rlike("^[A-Za-z]{3} \\d{2},\\d{4}$"), substring(col("FechaTransaccion"), -4, 4).cast("int"))
    .otherwise(substring(col("FechaTransaccion"), 1, 4).cast("int"))
)
# Filtrar los movimientos realizados desde 2013
movimientos_desde_2013 = movimientosAnio.filter(col("AnioTransaccion") >= 2013)
cantidad_movimientos_desde_2013 = movimientos_desde_2013.count()
print(cantidad_movimientos_desde_2013)
print(movimientosAnio.agg(min("AnioTransaccion")).collect()[0][0])

204292
2013


El análisis muestra que si hay movimientos registrados desde el año 2013 con un total de 204,292 movimientos, lo cual no coincide con los 236,668 movimientos indicados por el negocio. Esta discrepancia puede deberse a varias razones, como errores en los datos proporcionados, diferencias en la definición de lo que constituye un movimiento válido, es se  hace evidente problemas en la calidad de datos (fechas mal formateadas o registros duplicados)

In [40]:
# Columnas clave para identificar duplicados
key_columns = ["TransaccionProductoID", "ProductoID", "FechaTransaccion"]
# Contar las ocurrencias de cada combinación de las columnas clave
duplicates = movimientos.groupBy(key_columns).count()
# Registros que tienen más de una ocurrencia
duplicates = duplicates.filter(col("count") > 1)
duplicates.show()
print(duplicates.count())

+---------------------+----------+--------------------+-----+
|TransaccionProductoID|ProductoID|    FechaTransaccion|count|
+---------------------+----------+--------------------+-----+
|               280240|       212|2015-11-18 12:00:...|    2|
|               284037|       210|2015-12-01 12:00:...|    2|
|               183029|       157|2014-12-19 12:00:...|    2|
|               228081|       168|2015-05-25 12:00:...|    2|
|               235099|       104|2015-06-16 12:00:...|    2|
|               144748|        60|2014-07-25 12:00:...|    2|
|               230367|        29|2015-06-01 12:00:...|    2|
|               262693|       105|2015-09-18 12:00:...|    2|
|               279132|        50|2015-11-13 12:00:...|    2|
|               322005|        73|2016-04-14 12:00:...|    2|
|               150870|        39|2014-08-18 12:00:...|    2|
|               204830|       131|2015-03-09 12:00:...|    2|
|               324226|       109|2016-04-21 12:00:...|    2|
|       

El negocio esperaba un total de 236,668 movimientos desde 2013, mientras que el análisis muestra 204,292 movimientos únicos. La diferencia de 32,376 movimientos podría estar explicada parcialmente por los 30,633 duplicados, aunque no completamente. Además de los duplicados, puede haber otros problemas en los datos, como registros faltantes o mal categorizados.
Se informará al negocio sobre las incosistencias en los datos y estandarizaremos los formatos de fecha.

#### Regla 3: Los movimientos están relacionados con 228.265 clientes

In [16]:
clientes_unicos = movimientos.select("ClienteID").distinct().count()
print(f"La cantidad de clientes únicos relacionados con los movimientos es: {clientes_unicos}")

La cantidad de clientes únicos relacionados con los movimientos es: 664


La discrepancia significativa entre los 664 clientes únicos encontrados y los 228,265 clientes mencionados por el negocio indica la necesidad de una revisión exhaustiva y posible corrección de los datos.

In [17]:
print(movimientos.filter(col("ClienteID").isNull() | (col("ClienteID") == "")).count())
print(movimientos.filter(col("ClienteID") == 0).count())

0
7156


La presencia de 7,156 movimientos con ClienteID igual a 0 indica que estos registros no están correctamente asociados a clientes específicos, donde 0 podría estar siendo usado como un valor predeterminado o marcador de datos faltantes. Debe ser aclarado por la empresa para asegurar la precisión en el análisis de los movimientos y el comportamiento de los clientes.

#### Regla 4: El formato de fechas manejado es YYYY-MM-DD HH:MM:SS

In [18]:
regex = "[0-2]\d{3}-(0[1-9]|1[0-2])-(0[1-9]|[1-2][0-9]|3[0-1])"
cumplenFormato = movimientos.filter(movimientos["FechaTransaccion"].rlike(regex))
print("Número total de ordenes: "+str(movimientos.count())+", número de ordenes con formato correcto: "+str(cumplenFormato.count()))

Número total de ordenes: 204292, número de ordenes con formato correcto: 140038


Registros que no cumplen con el formato correcto

In [19]:
noCumplenFormato = movimientos.filter(~movimientos["FechaTransaccion"].rlike(regex))
noCumplenFormato.count(), noCumplenFormato.select(col("FechaTransaccion")).show()

+----------------+
|FechaTransaccion|
+----------------+
|     Apr 25,2014|
|     Dec 10,2015|
|     Dec 04,2015|
|     Dec 23,2015|
|     Jul 27,2015|
|     Sep 15,2014|
|     Aug 04,2015|
|     Feb 23,2015|
|     May 01,2015|
|     Jan 08,2016|
|     Mar 26,2014|
|     Jul 31,2015|
|     Sep 02,2014|
|     Mar 15,2016|
|     May 28,2016|
|     Sep 09,2015|
|     May 23,2014|
|     Aug 20,2014|
|     Jan 21,2015|
|     Dec 29,2015|
+----------------+
only showing top 20 rows



(64254, None)

#### Regla 5: Actualmente tenemos 13 proveedores

In [20]:
print(movimientos.select("ProveedorID").distinct().count())

4


### Análisis descriptivo

In [21]:
movimientos.summary().show()

+-------+---------------------+------------------+-------------------+------------------+-----------------+-----------------+------------------+--------------------+-----------------+
|summary|TransaccionProductoID|        ProductoID|  TipoTransaccionID|         ClienteID|        InvoiceID|      ProveedorID|   OrdenDeCompraID|    FechaTransaccion|         Cantidad|
+-------+---------------------+------------------+-------------------+------------------+-----------------+-----------------+------------------+--------------------+-----------------+
|  count|               204292|            204292|             204292|            204292|           204292|           204292|            204292|              204292|           204292|
|   mean|   212458.04047637695|110.70090850351458| 10.035253460732676| 517.3252941867523|42957.26929590978|4.951898734177215|1345.9973277074544|                null|719.4997650421946|
| stddev|    71352.37579752573| 63.49014746219581|0.18563716955046372|353.501369

El análisis de la tabla de movimientos revela lo siguiente: los TransaccionProductoID van desde 89,146 hasta 336,251, y los ProductoID desde 1 hasta 227. Los TipoTransaccionID varían entre 10 y 12, mostrando que solo ciertos tipos de transacciones están presentes. Los ClienteID van de 0 a 1061, con 7,156 registros teniendo ClienteID igual a 0, lo cual es problemático. Los InvoiceID varían de 0 a 70,510, mientras que los ProveedorID van de 0 a 7, con solo 4 proveedores únicos, discrepando con los 13 mencionados por el negocio. Las OrdenDeCompraID varían de 0 a 999.

## Análisis de calidad de datos
La calidad de los datos se valida en cuanto a la idoneidad de:
- Completitud: Datos que no existen o no se conocen
- Unicidad (Duplicidad): Datos que son idénticos en diferentes registros o registros con valores idénticos en atributos en los que no debería ocurrir
- Consistencia: Esta medida se define por la validez o integridad del dato, en la definición de su estructura, al interior de una fila o entre diferentes filas de la misma fuente o de diferentes fuentes. El manejo de unidades en los datos y el significado del dato son generadores de inconsistencias.
- Validez: a nivel de formato y de sentido de los datos más alla de sus valores


### Completitud

#### Verificación de valores nulos o vacíos

In [41]:
# Calcular el número total de filas
total_filas = movimientos.count()

# Calcular el número de valores incompletos (nulos o vacíos) por columna
incompletos_por_columna = movimientos.select(
    [count(when(col(c).isNull() | (col(c) == ""), c)).alias(c) for c in movimientos.columns]
).collect()[0]

# Calcular la completitud por columna
completitud_por_columna = {c: (1 - (incompletos_por_columna[c] / total_filas)) * 100 for c in movimientos.columns}

for c, comp in completitud_por_columna.items():
    print(f"Completitud de la columna {c}: {comp:.3f}%")

Completitud de la columna TransaccionProductoID: 100.00%
Completitud de la columna ProductoID: 100.00%
Completitud de la columna TipoTransaccionID: 100.00%
Completitud de la columna ClienteID: 100.00%
Completitud de la columna InvoiceID: 100.00%
Completitud de la columna ProveedorID: 3.48%
Completitud de la columna OrdenDeCompraID: 3.48%
Completitud de la columna FechaTransaccion: 100.00%
Completitud de la columna Cantidad: 100.00%



#### Comprobación

In [38]:
total_filas = movimientos.count()
# Verificar la relación entre TipoTransaccionID y valores vacíos en ProveedorID y OrdenDeCompraID
proveedor_vacio_por_tipo = movimientos.filter(col("ProveedorID") == "").groupBy("TipoTransaccionID").count()
orden_vacio_por_tipo = movimientos.filter(col("OrdenDeCompraID") == "").groupBy("TipoTransaccionID").count()

proveedor_vacio_por_tipo.show()
orden_vacio_por_tipo.show()

+-----------------+------+
|TipoTransaccionID| count|
+-----------------+------+
|               12|    46|
|               10|197136|
+-----------------+------+

+-----------------+------+
|TipoTransaccionID| count|
+-----------------+------+
|               12|    46|
|               10|197136|
+-----------------+------+



### Unicidad

#### Verificación de registros duplicados

In [23]:
# Columnas para identificar duplicados
key_columns = ["TransaccionProductoID", "ProductoID", "TipoTransaccionID", "ClienteID", "InvoiceID", "ProveedorID", "OrdenDeCompraID", "FechaTransaccion", "Cantidad"]
# Contar las ocurrencias de cada combinación de las columnas
duplicates = movimientos.groupBy(key_columns).count()
# Se filtrar aquellos registros que tienen más de una ocurrencia
duplicates = duplicates.filter(col("count") > 1)
duplicates.show()
# Total de duplicados
total_duplicates = duplicates.count()
print(f"Total de registros duplicados: {total_duplicates}")

+---------------------+----------+-----------------+---------+---------+-----------+---------------+--------------------+--------+-----+
|TransaccionProductoID|ProductoID|TipoTransaccionID|ClienteID|InvoiceID|ProveedorID|OrdenDeCompraID|    FechaTransaccion|Cantidad|count|
+---------------------+----------+-----------------+---------+---------+-----------+---------------+--------------------+--------+-----+
|               270985|       216|               10|    510.0|  56780.0|           |               |2015-10-16 12:00:...|   -10.0|    2|
|               118236|       119|               10|    425.0|  24762.0|           |               |2014-04-23 12:00:...|   -10.0|    2|
|               265915|       152|               10|    911.0|  55734.0|           |               |2015-09-29 12:00:...|   -10.0|    2|
|               261670|        42|               10|    995.0|  54844.0|           |               |2015-09-15 12:00:...|   -10.0|    2|
|               218629|        65|       

### Consistencia

#### Verificación consistencia estructural

In [24]:
# Tipos de datos de las columnas
movimientos.printSchema()
# Verificar la longitud de los valores en columnas de tipo string
from pyspark.sql.functions import length
string_columns = [field.name for field in movimientos.schema.fields if field.dataType == StringType()]
movimientos.select([length(col(c)).alias(c) for c in string_columns]).summary().show()

root
 |-- TransaccionProductoID: integer (nullable = true)
 |-- ProductoID: integer (nullable = true)
 |-- TipoTransaccionID: integer (nullable = true)
 |-- ClienteID: double (nullable = true)
 |-- InvoiceID: double (nullable = true)
 |-- ProveedorID: string (nullable = true)
 |-- OrdenDeCompraID: string (nullable = true)
 |-- FechaTransaccion: string (nullable = true)
 |-- Cantidad: double (nullable = true)

+-------+-------------------+-------------------+------------------+
|summary|        ProveedorID|    OrdenDeCompraID|  FechaTransaccion|
+-------+-------------------+-------------------+------------------+
|  count|             204292|             204292|            204292|
|   mean|0.10440937481643922|0.19980713880132359|21.967673721927437|
| stddev| 0.5498438749162375| 1.0553991369448272|7.4292114553489785|
|    min|                  0|                  0|                11|
|    25%|                  0|                  0|                11|
|    50%|                  0|      

#### Verificación consistencia de contenido

In [42]:
# Se define los tipos de transacción permitidos para cantidades negativas
tipos_transaccion_permitidos = [2, 4, 6, 8, 10, 12, 13]
# Filtrar registros con cantidad negativa que no corresponden a los tipos de transacción permitidos
cantidad_negativa_inconsistente = movimientos.filter((col("Cantidad") < 0) & (~col("TipoTransaccionID").isin(tipos_transaccion_permitidos))).count()
print(f"Cantidad de registros con cantidad negativa inconsistente: {cantidad_negativa_inconsistente}")

Cantidad de registros con cantidad negativa inconsistente: 0


### Validez

#### Verificación valores en rangos específicos

In [29]:
# Cantidades dentro de los rangos esperados no más de 50 millones por transacción
cantidades_invalidas = movimientos.filter((col("Cantidad") < -50000000) | (col("Cantidad") > 50000000)).count()
print(f"Cantidad de registros con cantidades fuera del rango permitido: {cantidades_invalidas}")

Cantidad de registros con cantidades fuera del rango permitido: 0


## Entendimiento de datos

### Analizar la correlación lineal

In [35]:
# Calcular la correlación entre TipoTransaccionID y Cantidad
correlacion = movimientos.select(corr("TipoTransaccionID", "Cantidad")).collect()[0][0]
print(f"Correlación entre TipoTransaccionID y Cantidad: {correlacion}")

Correlación entre TipoTransaccionID y Cantidad: 0.8336502987666313


## Conclusiones/resultados

**Conclusiones generales:**

El análisis de la tabla ....

**Conclusiones de reglas de negocio:**

- 
- 
- 
- 
- 

**Conclusiones de calidad:**

- 
- 
- 
- 

**Sección de correlación**





## Para la organización

**Preguntas y aclaraciones**
- 
- 
- 
- 
- 
- 

**Problemas de calidad y alternativas para la limpieza**

- 
- 
- 